In [22]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from IPython.display import display
pd.options.display.max_columns = None
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_absolute_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import GRU
from keras.layers import Dropout
from keras.layers import Bidirectional
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from scipy import stats

In [23]:
df_train = pd.read_csv('soan95train.csv', index_col=None)
df_test = pd.read_csv('soan95test.csv', index_col=None)

In [24]:
X_train = df_train['rain'].astype('float64')
y_train = df_train['basin'].astype('float64')
X_test = df_test['rain'].astype('float64')
y_test = df_test['rain'].astype('float64')

In [25]:
min_max_scaler = MinMaxScaler(feature_range=(0, 1))
X_train_scaled = min_max_scaler.fit_transform(np.array(X_train).reshape(-1, 1))
X_test_scaled = min_max_scaler.transform(np.array(X_test).reshape(-1, 1))

In [26]:
train_1s = X_train_scaled.reshape(X_train_scaled.shape[0],1,X_train_scaled.shape[1])

In [27]:
train_1s.shape

(1826, 1, 1)

In [28]:
model_sj = Sequential()
model_sj.add(Bidirectional(GRU(50, activation='relu', return_sequences=True, input_shape=(1, 1))))
model_sj.add(Dropout(0.2))
model_sj.add(Bidirectional(GRU(100, activation='relu', return_sequences=False)))
model_sj.add(Dropout(0.2))
model_sj.add(Dense(1, activation = "linear"))
model_sj.compile(optimizer='adam', loss='mae')

In [29]:
%%time
model_1s.fit(train_1s, np.array(y_train), epochs=400, verbose=1)

Epoch 1/400
1826/1826 [==============================] - 2s 895us/step - loss: 26.1740
Epoch 2/400
1826/1826 [==============================] - 1s 483us/step - loss: 26.1422
Epoch 3/400
1826/1826 [==============================] - 1s 534us/step - loss: 26.1517
Epoch 4/400
1826/1826 [==============================] - 1s 526us/step - loss: 26.1351
Epoch 5/400
1826/1826 [==============================] - 1s 493us/step - loss: 26.1504
Epoch 6/400
1826/1826 [==============================] - 1s 454us/step - loss: 26.1620
Epoch 7/400
1826/1826 [==============================] - 1s 445us/step - loss: 26.1430
Epoch 8/400
1826/1826 [==============================] - 1s 497us/step - loss: 26.1515
Epoch 9/400
1826/1826 [==============================] - 1s 461us/step - loss: 26.1401
Epoch 10/400
1826/1826 [==============================] - 1s 463us/step - loss: 26.1348
Epoch 11/400
1826/1826 [==============================] - 1s 480us/step - loss: 26.1298
Epoch 12/400
1826/1826 [=================

In [30]:
X_test_reshape = X_test_scaled.reshape(X_test_scaled.shape[0], 1, X_test_scaled.shape[1])

In [31]:
result = model_1s.predict(X_test_reshape)

In [32]:
mean_absolute_error(y_test, result)

12.398501296121568

In [33]:
pd.DataFrame(result).to_csv('gru.csv', index=False)